In [4]:
# Load packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [5]:
# Set more columns viewable
pd.set_option('display.max_columns',200)

In [6]:
#Load dataset and convert data points to same type
train = pd.read_csv('train.csv')
train.info()
train.iloc[:,1:] = train.iloc[:,1:].astype('int')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: float64(1845), int64(3147), object(1)
memory usage: 169.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: int32(4992), object(1)
memory usage: 84.9+ MB


In [7]:
train.head()

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,822e49b95,316b978cd,d04e16aed,5d5c5ce6d,ec863cb52,11ad148bd,ea18d720e,408d86ce9,69d8b4020,b0868a049,80b14398e,ecdef52b2,c36bcacc5,151d318cd,ab0cddb90,2d6bd8275,dfdf4b580,c87722678,a8c320153,7df8788e8,a04684f1f,fa977f17b,feed9d437,645b47cde,7298ca1ef,d80abf8bc,654dd8a3b,7fc39583c,8c94b6675,e421c414e,5b91580ee,964335fdf,5a86cabd0,6cb207ac9,c1bc828da,e90ed19da,c4d38135f,968b01ca3,26b423c42,0656586a4,b4ced4b7a,0b8e10df6,603e85eb7,605728195,4e8196700,f115e74c0,9ca0eee11,472cd130b,ee7e4581d,3e0e55648,a939664aa,b0f5ed414,a72e0bf30,e662f1672,54b1c1bc0,ec1425047,78a879b5d,bd550871c,828b327a6,3268914c7,...,e369704a1,7df9efba5,f5723deba,7fd35c77a,9d9e518a3,99258443a,1d81e197a,831a9e087,de471ea65,8781e4b91,dc5ba677f,df3ac443c,79203fbab,f1ab19e7a,300398f1c,6eefca12e,22bf44001,19915a6d3,20a325694,8afae1db8,c283d4609,11114a47a,b961b0d59,103b8a033,43d70cc4d,09184c121,08c089775,572d36d31,1de239e78,d7568383a,8d7bfb911,5964f1856,6c7bf200b,b3058e9ba,108fe14ef,dcfcddf16,5fad07863,b904b8345,408021ef8,ef30f6be5,e29d22b59,a19b05919,e849e3ae0,05f11f48f,49f11e712,c959d372a,b625fe55a,f642213a6,6df033973,1834f29f5,31e434a58,e431708ff,fb42abc0d,88263a031,ab7764ead,a380d5301,46ee7f2c8,2e1287e41,23310aa6f,9dc52515a,969caa87a,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,000d6aaf2,38000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1300000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1266666,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1100000,0,0,0,0,0,14800000,0,1200000,0,0,0,0,0,0,0,4000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,000fbd867,600000,0,0,0,0,0,0,0,0,2200000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0027d6b71,10000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0028cbf45,2000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,002a68644,14400000,0,0,0,0,0,0,0,0,2000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Columns: 4993 entries, ID to 9fc776466
dtypes: int32(4992), object(1)
memory usage: 84.9+ MB


## Drawing inspiration from Giba

In [168]:
cols = [
    "f190486d6","58e2e02e6","eeb9cd3aa","9fd594eec","6eef030c1","15ace8c9f",
    "fb0f5dbfe","58e056e12","20aa07010","024c577b9","d6bb78916",
    "b43a7cfd5","58232a6fb"
]
rows = np.array([2072,3493,379,2972,2367,4415,2791,3980,194,1190,3517,811,4444])-1


## Train is a time series data set with target being an n-2 lag with f190486d6 being the last known timestamp

In [169]:
sort_train = train.loc[rows,cols]

In [170]:
sort_train

,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb
2071,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444
3492,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166
378,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000
2971,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666
2366,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000
4414,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0
2790,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000
3979,1359000,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461
193,60000,1359000,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200
1189,12000000,60000,1359000,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000


In [171]:
train2 = train.iloc[rows,2:]
train2

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,822e49b95,316b978cd,d04e16aed,5d5c5ce6d,ec863cb52,11ad148bd,ea18d720e,408d86ce9,69d8b4020,b0868a049,80b14398e,ecdef52b2,c36bcacc5,151d318cd,ab0cddb90,2d6bd8275,dfdf4b580,c87722678,a8c320153,7df8788e8,a04684f1f,fa977f17b,feed9d437,645b47cde,7298ca1ef,d80abf8bc,654dd8a3b,7fc39583c,8c94b6675,e421c414e,5b91580ee,964335fdf,5a86cabd0,6cb207ac9,c1bc828da,e90ed19da,c4d38135f,968b01ca3,26b423c42,0656586a4,b4ced4b7a,0b8e10df6,603e85eb7,605728195,4e8196700,f115e74c0,9ca0eee11,472cd130b,ee7e4581d,3e0e55648,a939664aa,b0f5ed414,a72e0bf30,e662f1672,54b1c1bc0,ec1425047,78a879b5d,bd550871c,828b327a6,3268914c7,21e0e6ae3,ec73030f6,...,e369704a1,7df9efba5,f5723deba,7fd35c77a,9d9e518a3,99258443a,1d81e197a,831a9e087,de471ea65,8781e4b91,dc5ba677f,df3ac443c,79203fbab,f1ab19e7a,300398f1c,6eefca12e,22bf44001,19915a6d3,20a325694,8afae1db8,c283d4609,11114a47a,b961b0d59,103b8a033,43d70cc4d,09184c121,08c089775,572d36d31,1de239e78,d7568383a,8d7bfb911,5964f1856,6c7bf200b,b3058e9ba,108fe14ef,dcfcddf16,5fad07863,b904b8345,408021ef8,ef30f6be5,e29d22b59,a19b05919,e849e3ae0,05f11f48f,49f11e712,c959d372a,b625fe55a,f642213a6,6df033973,1834f29f5,31e434a58,e431708ff,fb42abc0d,88263a031,ab7764ead,a380d5301,46ee7f2c8,2e1287e41,23310aa6f,9dc52515a,969caa87a,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
2071,0,0,0,0,0,0,0,0,440000,0,4434140,0,0,7822000,0,0,0,3160000,0,0,0,0,80000,812666,0,0,0,1674000,5240000,3450000,0,1964666,2002166,0,1600000,0,0,0,1166666,5422000,2831000,0,0,1000000,0,16948000,0,0,2002166,1033000,0,0,0,5300000,0,0,0,0,0,0,400000,0,0,0,2400000,0,803538,0,0,0,0,556000,0,0,0,0,0,6740000,4434140,7640000,0,0,812666,4070000,2002166,2000000,0,0,4000000,0,720000,0,2002166,1674000,0,1612000,400000,0,0,0,...,0,0,0,1492000,0,0,4000000,0,0,1775333,0,0,0,0,400000,0,0,6568857,0,0,0,1470000,2865694,0,0,0,0,0,0,17500000,0,0,0,0,0,520000,0,0,0,1964666,0,2831000,0,1674000,0,0,2831000,1674000,3822750,1200000,0,0,0,0,0,0,0,1500000,1563411,0,0,6902000,4000000,0,0,0,825000,0,800000,0,3822750,0,1470000,0,0,0,0,0,0,0,346000,0,3400000,2992315,0,10600000,1166666,0,0,0,1674000,600000,3822750,40770000,0,0,0,0,3000000,14650000
3492,0,0,0,0,0,0,0,0,0,0,4434140,0,0,7822000,0,0,0,3160000,0,0,0,0,80000,0,0,0,0,1674000,0,3007600,0,273000,2002166,0,1066666,0,0,732000,1166666,5422000,2831000,0,0,0,0,0,0,0,4119200,1033000,0,0,0,2361500,0,0,0,0,0,0,400000,0,0,0,2400000,0,4184200,0,0,0,0,0,0,0,0,0,0,0,4434140,360000,0,0,812666,0,2002166,2000000,0,0,4000000,0,0,0,2002166,1674000,0,1612000,400000,0,0,0,...,162000,0,0,0,0,0,4000000,0,0,1775333,0,0,0,0,400000,0,0,6568857,0,0,0,1470000,2865694,0,0,0,0,0,0,660000,300000,0,0,0,0,2836333,355000,0,0,1964666,0,2831000,0,1674000,0,0,2831000,1674000,3822750,0,0,0,0,0,0,0,0,1500000,1563411,0,0,6902000,0,178000,0,80000,148666,0,0,17538666,3822750,1600000,1470000,0,0,0,0,0,0,0,180000,0,3044000,2992315,0,3920000,1166666,0,0,0,1674000,0,3822750,40770000,0,0,0,0,3000000,14650000
378,0,0,0,0,0,0,0,0,1600000,22000,4434140,0,0,7822000,0,0,0,3160000,0,0,0,0,80000,0,0,0,0,1674000,0,572500,0,210000,2002166,0,0,0,0,0,1166666,5422000,2831000,0,0,0,0,0,0,0,34000,1033000,0,0,0,2000000,0,0,84000,0,0,0,400000,0,0,0,2400000,0,1446333,0,0,0,0,0,0,0,0,0,0,0,12228500,0,0,0,812666,0,4119200,2000000,0,0,4000000,0,0,0,2002166,1674000,0,1612000,400000,0,0,0,...,0,0,0

In [172]:
new_cols = cols 
new_cols

['f190486d6',
 '58e2e02e6',
 'eeb9cd3aa',
 '9fd594eec',
 '6eef030c1',
 '15ace8c9f',
 'fb0f5dbfe',
 '58e056e12',
 '20aa07010',
 '024c577b9',
 'd6bb78916',
 'b43a7cfd5',
 '58232a6fb']

In [207]:
def col_sort(sort_df, df, modify = 0):
    head_key = sort_df.iloc[:-1,-1]
    tail_key = sort_df.iloc[1:,0]
    print(sort_df.shape)
    count = 0
    if modify == 0:
        while True:

            for col in df.columns:
                if col in sort_df.columns:
                    continue
                # Head key match, concat new column right side
                elif np.all(head_key.values == df[col].iloc[1:].values):
                    sort_df = pd.concat([sort_df,df[col]],axis = 1)
                    print('found head')
                    head_key = sort_df.iloc[:-1,-1]

                    count += 1

                # Tail key match, concat new column left side
                elif np.all(tail_key.values == df[col].iloc[:-1].values):
                    sort_df = pd.concat([df[col],sort_df], axis = 1)
                    print('found tail')
                    tail_key = sort_df.iloc[1:,0]

                    count += 1
            if count == 0:
                break
            else:
                count = 0
    if modify == 1:
        head_key = sort_df.iloc[0,-1]
        tail_key = sort_df.iloc[1,0]
        while True:

            for col in df.columns:
                if col in sort_df.columns:
                    continue
                # Head key match, concat new column right side
                elif np.all(head_key == df[col].iloc[1]):
                    sort_df = pd.concat([sort_df,df[col]],axis = 1)
                    print('found head')
                    head_key = sort_df.iloc[0,-1]
                    count += 1
                # Tail key match, concat new column left side
                elif np.all(tail_key == df[col].iloc[0]):
                    sort_df = pd.concat([df[col],sort_df], axis = 1)
                    print('found tail')
                    tail_key = sort_df.iloc[1,0]
                    count += 1
            if count == 0:
                break
            else:
                count = 0
    print(sort_df.shape)
    new_cols = sort_df.columns
    return sort_df

In [174]:
sorted_train = col_sort(sort_train,train2)
sorted_train

(13, 13)
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
found head
(13, 40)


,f190486d6,58e2e02e6,eeb9cd3aa,9fd594eec,6eef030c1,15ace8c9f,fb0f5dbfe,58e056e12,20aa07010,024c577b9,d6bb78916,b43a7cfd5,58232a6fb,1702b5bf0,324921c7b,62e59a501,2ec5b290f,241f0f867,fb49e4212,66ace2992,f74e8f13d,5c6487af1,963a49cdc,26fc93eb7,1931ccfdd,703885424,70feb1494,491b9ee45,23310aa6f,0572565c2,6619d81fc,fc99f9426,1db387535,0ff32eb98,91f701ba2,adb64ff71,c5a231d81,e176a204a,190db8488,c47340d97
2071,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
3492,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
378,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
2971,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
2366,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
4414,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
2790,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
3979,1359000,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
193,60000,1359000,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411
1189,12000000,60000,1359000,5450500,280000,2352551,160000,3513333,310000,0,1477600,1586888,75000,3147200,466461,1600000,0,440000,3076666,550000,247166,296444,834800,115636,540000,1015000,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411,1563411


few = []
for col in train.columns:
    if len(train[col].unique()) < 2:
        few.append(col)

In [175]:
new_cols = sorted_train.columns
new_cols

Index(['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1',
       '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
       'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b',
       '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992',
       'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd',
       '703885424', '70feb1494', '491b9ee45', '23310aa6f', '0572565c2',
       '6619d81fc', 'fc99f9426', '1db387535', '0ff32eb98', '91f701ba2',
       'adb64ff71', 'c5a231d81', 'e176a204a', '190db8488', 'c47340d97'],
      dtype='object')

In [176]:
train3 = train.T.iloc[2:,:]
train3 = train3.loc[new_cols, :]
train3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,...,4359,4360,4361,4362,4363,4364,4365,4366,4367,4368,4369,4370,4371,4372,4373,4374,4375,4376,4377,4378,4379,4380,4381,4382,4383,4384,4385,4386,4387,4388,4389,4390,4391,4392,4393,4394,4395,4396,4397,4398,4399,4400,4401,4402,4403,4404,4405,4406,4407,4408,4409,4410,4411,4412,4413,4414,4415,4416,4417,4418,4419,4420,4421,4422,4423,4424,4425,4426,4427,4428,4429,4430,4431,4432,4433,4434,4435,4436,4437,4438,4439,4440,4441,4442,4443,4444,4445,4446,4447,4448,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
f190486d6,1866666,0,0,2000000,0,0,10000,0,3333333,7500000,0,11200000,0,3300000,0,0,20000,700000,0,2000000,700000,800000,3587000,0,0,19000,0,0,500000,0,0,0,0,0,0,1400000,0,0,0,7000000,20000000,0,0,6000000,17057333,16000000,0,0,1200000,0,0,0,0,4000000,0,0,0,0,193333,0,1021000,0,0,0,6050000,0,6000000,4000000,8120666,10000000,13900000,0,0,0,0,0,10000,0,0,0,460000,0,0,92000,0,0,0,0,4000000,0,0,7100000,0,0,4000000,32000000,4000000,0,0,0,...,0,0,0,0,0,0,80000,0,0,0,2900000,1092500,6090000,0,0,0,13866666,0,777750,0,10000,30000000,18062333,0,8978615,448666,0,0,0,0,0,2000000,12350000,0,0,0,0,400000,0,7621333,0,0,0,0,0,0,0,0,20000,0,0,6000000,20000000,1060000,0,280000,0,0,0,0,0,0,0,1943333,0,0,2000000,9046666,5400000,0,400000,0,8266666,0,16000000,98000,0,0,5000000,8000000,0,0,0,0,814800,1000000,0,0,0,2000000,0,0,0,0,4000000,0,242666,0,200000,0
58e2e02e6,12066666,2850000,0,0,0,0,4000,0,3925333,0,8300000,0,0,0,0,0,200000,0,2324000,0,0,0,11515142,3665000,0,0,0,52820000,190000,0,0,0,0,0,0,0,0,20000,0,4300000,20000000,0,0,0,3090000,7300000,3200000,0,0,0,0,8000000,0,0,0,0,0,0,0,0,2900000,0,0,0,0,2256000,0,0,10360500,15000000,10000000,116000000,0,0,43000000,1900000,0,17000000,0,0,0,0,0,0,0,0,0,0,2000000,0,0,2400000,26666,5000000,0,28000000,6400000,0,0,0,...,0,0,36000,67500,12580000,0,30000,0,0,6000,11000000,766000,0,0,0,0,0,0,1110333,280000,0,0,500000,0,0,406000,0,0,0,0,0,0,0,0,0,0,0,0,11500000,0,5000000,0,0,0,0,0,0,0,60000,0,0,80000,10000000,7279000,0,2352551,0,0,0,0,0,0,10000,0,35000000,0,0,7800000,1066666,0,0,0,3225000,0,0,1000000,10800000,0,0,0,5555555,4000,0,0,1878571,220000,0,0,0,6500000,1600000,0,0,0,0,680000,0,4640000,0,0
eeb9cd3aa,700000,2225000,0,0,0,2800000,0,0,4000000,0,0,0,0,8800000,0,0,400000,0,2580000,0,0,12797000,12533714,0,0,0,0,1630000,364444,9300000,0,0,7000000,0,0,0,0,0,0,2500000,20000000,0,0,10000000,2590000,12000,0,0,1400000,0,0,7000000,0,6000000,0,2000000,0,0,0,0,0,0,0,0,0,0,0,0,5764571,0,7012000,0,12500000,0,50000000,0,69000,5700000,0,0,0,0,0,13000,43440000,0,0,0,0,0,0,1433333,20000,0,4500000,30000000,0,3500000,0,0,...,0,20000,0,0,0,2000000,60000,0,0,0,20000000,1988666,2250000,0,0,0,0,100000,1427333,6000000,0,0,12886500,0,2116666,609200,0,0,0,0,0,0,47000000,0,0,0,0,448666,0,0,0,0,0,0,0,0,0,0,10000,0,0,200000,0,1859333,0,160000,407500,0,5866666,0,320000,13333333,0,2453200,0,6800000,500000,6000000,1066666,0,0,0,3500000,11000000,4500000,0,0,400000,0,0,3250000,0,0,0,500000,0,1600000,0,0,4000000,0,0,0,0,0,3191666,37333,2597000,0,0
9fd594eec,600000,1800000,0,0,0,17000000,30000,65000000,0,0,0,0,0,0,0,72000,0,788000,0,0,0,0,10849666,600000,0,0,0,0,0,8000000,0,0,2000000,0,1582000,0,0,1530000,0,0,20000000,0,1650000,0,2575000,0,0,0,700000,0,0,0,0,4000000,0,0,0,0,0,26356000,1635600,0,0,0,950000,0,15000000,0,0,0,2026000,0,0,0,47000000,0,50000,4000000,0,5000000,0,58000,0,0,0,0,0,0,2000000,0,0,3812000,50000,0,0,27500000,7000000,0,0,0,...,500000,0,0,0,0,0,130000,0,0,500000,1000000,1545200,7700000,0,0,0,0,40000,1134000,200000,0,0,1387142,0,9300000,528666,0,0,3600000,0,0,0,10000000,0,0,0,0,406000,0,1364000,0,0,0,0,0,0,0,0,92500,0,0,80000,0,972000,200000,3513333,378666,0,2000000,0,0,0,0,0,0,2800000,64800,3200000,1066666,0,0,0,0,0,4464000,966000,0,0,5000000,0,1

## There is a repeating figure, 1.563412e+06, for what looks like every columns. Must be the end of relevant data entries. The figure is usually preceded by 1.015000e+06

In [177]:
trans_train = sorted_train.T
trans_train

,2071,3492,378,2971,2366,4414,2790,3979,193,1189,3516,810,4443
f190486d6,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800
58e2e02e6,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571
eeb9cd3aa,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000
9fd594eec,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000
6eef030c1,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000
15ace8c9f,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000
fb0f5dbfe,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500
58e056e12,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000
20aa07010,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551
024c577b9,3076666,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000


## We were able to find a new first column labeled 1383.

In [189]:
trans_train = col_sort(trans_train,train3, modify = 1)
trans_train

(40, 14)
found head
found head
found head
(40, 17)


,1383,2071,3492,378,2971,2366,4414,2790,3979,193,1189,3516,810,4443,3697,235,1382
f190486d6,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000,528666,609200
58e2e02e6,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000,528666
eeb9cd3aa,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000
9fd594eec,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800
6eef030c1,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571
15ace8c9f,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000
fb0f5dbfe,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000
58e056e12,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000
20aa07010,3076666,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000
024c577b9,550000,3076666,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500


## Some of the rows need to be rearranged , changed repeating values to 1 for easier visibility

In [191]:
trans_train.replace(to_replace = 1563411, value = 1)

,1383,2071,3492,378,2971,2366,4414,2790,3979,193,1189,3516,810,4443,3697,235,1382
f190486d6,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000,528666,609200
58e2e02e6,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000,528666
eeb9cd3aa,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000
9fd594eec,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800
6eef030c1,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571
15ace8c9f,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000
fb0f5dbfe,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000
58e056e12,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000
20aa07010,3076666,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000
024c577b9,550000,3076666,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500


In [195]:
trans_train.index
new_rows = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1',
       '15ace8c9f', 'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9',
       'd6bb78916', 'b43a7cfd5', '58232a6fb', '1702b5bf0', '324921c7b',
       '62e59a501', '2ec5b290f', '241f0f867', 'fb49e4212', '66ace2992',
       'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', '1931ccfdd',
       '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
       '6619d81fc', '1db387535', '0572565c2','fc99f9426',  '0ff32eb98', '91f701ba2',
       'adb64ff71', 'c5a231d81',  '190db8488', 'c47340d97']

In [198]:
trans_train = trans_train.loc[new_rows, :]

In [209]:
col_sort(trans_train,train3, modify = 1)

(40, 17)
(40, 17)


,1383,2071,3492,378,2971,2366,4414,2790,3979,193,1189,3516,810,4443,3697,235,1382
f190486d6,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000,528666,609200
58e2e02e6,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000,528666
eeb9cd3aa,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800,307000
9fd594eec,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571,814800
6eef030c1,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000,1878571
15ace8c9f,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000,500000
fb0f5dbfe,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000,12000000
58e056e12,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000,60000
20aa07010,3076666,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500,1359000
024c577b9,550000,3076666,440000,0,1600000,466461,3147200,75000,1586888,1477600,0,310000,3513333,160000,2352551,280000,5450500


In [210]:
train[train['58e2e02e6'] == 609200]

,ID,target,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,11d86fa6a,77c9823f2,8d6c2a0b2,4681de4fd,adf119b9a,cff75dd09,96f83a237,b8a716ebf,6c7a4567c,4fcfd2b4d,f3b9c0b95,71cebf11c,d966ac62c,68b647452,c88d108c9,ff7b471cd,d5308d8bc,0d866c3d7,bc3f77679,bd8f989f1,0eff5bf95,22ed6dba3,92b13ebba,c330f1a67,233c7c17c,2cb4d123e,eeac16933,87ffda550,822e49b95,316b978cd,d04e16aed,5d5c5ce6d,ec863cb52,11ad148bd,ea18d720e,408d86ce9,69d8b4020,b0868a049,80b14398e,ecdef52b2,c36bcacc5,151d318cd,ab0cddb90,2d6bd8275,dfdf4b580,c87722678,a8c320153,7df8788e8,a04684f1f,fa977f17b,feed9d437,645b47cde,7298ca1ef,d80abf8bc,654dd8a3b,7fc39583c,8c94b6675,e421c414e,5b91580ee,964335fdf,5a86cabd0,6cb207ac9,c1bc828da,e90ed19da,c4d38135f,968b01ca3,26b423c42,0656586a4,b4ced4b7a,0b8e10df6,603e85eb7,605728195,4e8196700,f115e74c0,9ca0eee11,472cd130b,ee7e4581d,3e0e55648,a939664aa,b0f5ed414,a72e0bf30,e662f1672,54b1c1bc0,ec1425047,78a879b5d,bd550871c,828b327a6,3268914c7,...,e369704a1,7df9efba5,f5723deba,7fd35c77a,9d9e518a3,99258443a,1d81e197a,831a9e087,de471ea65,8781e4b91,dc5ba677f,df3ac443c,79203fbab,f1ab19e7a,300398f1c,6eefca12e,22bf44001,19915a6d3,20a325694,8afae1db8,c283d4609,11114a47a,b961b0d59,103b8a033,43d70cc4d,09184c121,08c089775,572d36d31,1de239e78,d7568383a,8d7bfb911,5964f1856,6c7bf200b,b3058e9ba,108fe14ef,dcfcddf16,5fad07863,b904b8345,408021ef8,ef30f6be5,e29d22b59,a19b05919,e849e3ae0,05f11f48f,49f11e712,c959d372a,b625fe55a,f642213a6,6df033973,1834f29f5,31e434a58,e431708ff,fb42abc0d,88263a031,ab7764ead,a380d5301,46ee7f2c8,2e1287e41,23310aa6f,9dc52515a,969caa87a,00302fe51,1189ee335,ca04a07ca,f6f15ffa5,841704460,ea5ed6ff7,b1bb8eac3,8132d18b8,c24ea6548,cdfc2b069,2a879b4f7,6b119d8ce,98dea9e42,9f2471031,88458cb21,f40da20f4,7ad6b38bd,c901e7df1,8f55955dc,85dcc913d,5ca0b9b0c,eab8abf7a,8d8bffbae,2a1f6c7f9,9437d8b64,5831f4c76,2e84e09c5,d45fd5508,a165f5761,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
